## ABS data - Missing Values Analysis

This notebook details the process of dealing with missing values when joining our data to external data. 

Initially, we used the <b> Postcode </b> based SEFIA ABS data. However, we find that there are a significant amount of missing values (~17.1%) when we join it to the consumer data. Hence, we decided to <b> reduce the granularity from postcode to SA2 areas instead </b>. That is, we use SA2 ABS data instead of postcode ABS data. Although missing values still persist, there are substantially much less missing values (2.79% unmatched SA2 codes and 1.67% missing scores). This notebook will go through the process of dealing with these missing values.

In [1]:
# import libraries and constants
import sys
sys.path.append('../scripts/utils')
from constants import *

import pandas as pd
from pyspark.sql import SparkSession

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

23/10/20 11:13:00 WARN Utils: Your hostname, vanessas-MacBook-Pro-3.local resolves to a loopback address: 127.0.0.1; using 192.168.18.7 instead (on interface en0)
23/10/20 11:13:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/20 11:13:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/20 11:13:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# read raw data
abs_data_SA2 = pd.read_pickle(f"{RAW_DATA}abs_table_1_SA2.pkl")

### Analyze missing values inherent in the ABS data (before Joins)

In [4]:
rows_with_missing_values = abs_data_SA2[abs_data_SA2.isna().any(axis=1)]
rows_with_missing_values

,SA2_code,SA2_name,relative_SE_dis_score,relative_SE_dis_decile,relative_SE_ad_dis_score,relative_SE_ad_dis_decile,economic_resources_score,economic_resources_decile,education_occupation_score,education_occupation_decile,population
239,111031230,Newcastle Port - Kooragang,NaN,<NA>,NaN,<NA>,1033.0,7,943,4,31
607,127021521,Wetherill Park Industrial,NaN,<NA>,NaN,<NA>,NaN,<NA>,929,3,37
629,128021537,Royal National Park,NaN,<NA>,NaN,<NA>,NaN,<NA>,1063,8,42
729,205031092,Wilsons Promontory,NaN,<NA>,NaN,<NA>,NaN,<NA>,1119,9,16
737,205051099,Alps - West,NaN,<NA>,NaN,<NA>,NaN,<NA>,1058,8,23
836,208031184,Braeside,NaN,<NA>,NaN,<NA>,NaN,<NA>,1055,8,31
1763,403041082,Lonsdale,NaN,<NA>,NaN,<NA>,NaN,<NA>,882,1,23
1983,506031130,Welshpool,NaN,<NA>,NaN,<NA>,NaN,<NA>,915,2,16
2002,507011150,Bibra Industrial,NaN,<NA>,NaN,<NA>,NaN,<NA>,1108,9,21
2019,507031173,Kwinana Industrial,NaN,<NA>,NaN,<NA>,NaN,<NA>,953,4,24


Thirteen SA2 areas in the ABS data have scores missing, and we've noticed that these areas generally have very small populations. We believe that the absence of scores is likely due to the challenges faced by the ABS in collecting data from such small population samples. It's worth mentioning that typical SA2 areas have populations ranging from 3,000 to 25,000.

## Joining ABS data to Consumer Data

In order to join the ABS data to our consumer data, we require two mapping files:
1. Postcode to SA2 Mapping File 
     * Our consumer data only comes with Postcode whereas our ABS data only comes with SA2 areas.
2. SA2 2016 to SA2 2021 Correspondence File    
     * Our mapping file comes with the SA2 code in 2016 version whereas our ABS data comes with the 2021 version of SA2.



### Add SA2_2016 code to the ABS data
We start with joining the correspondence file to our ABS data first. We note that:
  * Some SA2 2016 areas are split into multiple SA2 2021 areas.    
  * Multiple SA2 2016 areas are also merged into a single SA2 2021 area.

In [5]:
# read SA2 correspondence
sa2_correspondence = pd.read_csv(f"{LANDING_DATA}sa2_correspondence.csv")
sa2_correspondence = sa2_correspondence[["SA2_MAINCODE_2016", "SA2_CODE_2021"]]

# join them together
abs_new = pd.merge(abs_data_SA2, sa2_correspondence, left_on='SA2_code', right_on='SA2_CODE_2021', how='left')
abs_new = abs_new.drop('SA2_CODE_2021', axis=1)
abs_new = abs_new.rename(columns={'SA2_code': 'SA2_code_2021', 'SA2_MAINCODE_2016': 'SA2_code_2016'})
abs_new.head(5)

,SA2_code_2021,SA2_name,relative_SE_dis_score,relative_SE_dis_decile,relative_SE_ad_dis_score,relative_SE_ad_dis_decile,economic_resources_score,economic_resources_decile,education_occupation_score,education_occupation_decile,population,SA2_code_2016
0,101021007,Braidwood,1024.0,6,1001.0,6,1027.0,7,1008,6,4343,101021007.0
1,101021008,Karabar,994.0,5,982.0,5,1000.0,5,967,5,8517,101021008.0
2,101021009,Queanbeyan,1010.0,5,998.0,6,945.0,3,1000,6,11342,101021009.0
3,101021010,Queanbeyan - East,1025.0,6,1015.0,6,969.0,4,1025,7,5085,101021010.0
4,101021012,Queanbeyan West - Jerrabomberra,1098.0,10,1107.0,9,1109.0,10,1080,8,12744,101021012.0


### Add SA2_2016 code to Consumer Data via Postcode

Next, we join the SA2-postcode mapping file to our consumer data. We note that:
  * A postcode can belong to multiple SA2 areas    
  * Multiple postcodes belong to a single SA2 area    
  * 3 postcodes cannot be mapped into their respective SA2 area value.

In [6]:
sa2_postcode_map = (pd.read_pickle(f"{RAW_DATA}SA2_code.pkl")).drop("state", axis=1)
tbl_consumer = pd.read_pickle(f"{CURATED_DATA}tbl_consumer.pkl")
tbl_consumer_sa2 = pd.merge(tbl_consumer, sa2_postcode_map, on='postcode', how='left')
tbl_consumer_sa2 = tbl_consumer_sa2.rename(columns={'SA2_code': 'SA2_code_2016'})
tbl_consumer_sa2.head(5)

,name,state,postcode,gender,consumer_id,has_title,user_id,SA2_code_2016
0,Yolanda Williams,WA,6935,Female,1195503,False,1,504031066.0
1,Mary Smith,NSW,2782,Female,179208,False,2,124011455.0
2,Jill Jones MD,NT,0862,Female,1194530,True,3,702021055.0
3,Jill Jones MD,NT,0862,Female,1194530,True,3,702051066.0
4,Jill Jones MD,NT,0862,Female,1194530,True,3,702021056.0


In [7]:
null_SA2_count = tbl_consumer_sa2['SA2_code_2016'].isnull().sum()
print("Number of missing values (total):", null_SA2_count)
missing_postcodes = tbl_consumer_sa2.loc[tbl_consumer_sa2['SA2_code_2016'].isnull(), 'postcode'].tolist()
print("These missing values all come from these unique postcodes:")
print(list(set(missing_postcodes)))

Number of missing values (total): 488
These missing values all come from these unique postcodes:
['3989', '6958', '2755']


### Join the Consumer data and ABS data via SA2 2016 code
Now, both datasets have a common column (SA2 2016) and hence, we can join them.

In [8]:
consumerxabs = pd.merge(tbl_consumer_sa2, abs_new, on='SA2_code_2016', how='left')
initial_count = len(consumerxabs)

# create new column SA3 that extracts the SA3 from SA2 (more details below)
consumerxabs['SA3_code'] = consumerxabs['SA2_code_2016'].apply(lambda x: str(x)[:5])

In [9]:
consumer_sa2 = tbl_consumer_sa2['SA2_code_2016'].unique()
abs_sa2 = abs_new['SA2_code_2016'].unique()
print(f"% of unmatched SA2 areas in consumer dataset: {len(set(consumer_sa2) - set(abs_sa2))/len(consumer_sa2)*100:.2f}%")

missing_values_count = max(consumerxabs.isna().sum())
print(f"% of missing values after join: {missing_values_count/len(consumerxabs)*100:.2f}%")

% of unmatched SA2 areas in consumer dataset: 2.79%
% of missing values after join: 1.67%


### Deal with Missing Data
We acknolwedge three sources of Missing Data:
  * Missing scores/deciles from ABS data for 13 SA2codes.
  * 3 postcodes cannot be mapped to SA2 codes because it is missing from our mapping file.
  * Some SA2 codes present in consumer data does not exist in the ABS data (or the 2016-2021 correspondence data)

Magnitude of Missing Data:
  * < 2% rows with missing values anywhere.

### Deal with missing scores/deciles in ABS data

Option 1: Remove rows - remove consumers coming from these SA2 areas.   

**Option 2**: Impute with median of the SA3 area the SA2 area belongs to. ✅        

A SA2 code is a 9-digit code. A 9-digit SA2 code is fully hierarchical, and comprises: State and Territory identifier, Statistical Area Level 4 (SA4) identifier, SA3 identifier, SA2 identifier.


For example, 503021041 Perth City would mean that
   * 5 is the state/territory identifier
   * 03 is the SA4 identifier
   * 02 is the SA3 identifier
   * 1041 is the SA2 identifier

Hence, we can impute the scores using the median SA3 score.    
eg. missing SA2 is 111031230, then we impute using median of 11103 area.

In reality, because these missing SA2 areas are areas with low populations, it is expected that there are also less number of transactions coming from these areas. Hence, imputing these values will not have a significant effect to the final analysis.

In [10]:
# impute with median of SA3
missing_sa2 = consumerxabs.loc[consumerxabs['relative_SE_dis_score'].isna() & ~consumerxabs['SA2_code_2021'].isna(), 'SA2_code_2021'].unique().tolist()
to_impute = ["relative_SE_dis_score", "relative_SE_dis_decile", "relative_SE_ad_dis_score", "relative_SE_ad_dis_decile",
    "economic_resources_score", "economic_resources_decile"]

for score in to_impute:
    # calculate the median score for each 'SA3_code' group
    median_scores = consumerxabs.groupby('SA3_code')[score].median().reset_index()
    
    # rename the column in median_scores to match the original column name
    median_scores.rename(columns={score: f'{score}_median'}, inplace=True)
    
    # merge the median_scores DataFrame back to the original DataFrame for rows with SA2 in 'missing_sa2'
    consumerxabs = consumerxabs.merge(median_scores, on='SA3_code', how='left')
    
    # update missing 'score' values for the selected rows with the corresponding median
    consumerxabs.loc[consumerxabs['SA2_code_2021'].isin(missing_sa2), score] = consumerxabs[f'{score}_median']
    
    # drop the temporary median column
    consumerxabs.drop(columns=f'{score}_median', inplace=True)

### Deal with the 3 postcodes that has missing SA2 code in the mapping file

**Option 1**: Remove rows ✅     
The 3 missing postcodes are 3989, 2755 and 6958. Searching for these postcodes on Auspost returns no result. It is also found that postcodes 2755 and 6958 are reserved for non-standard use (PO Boxes, competition mail, government departments, large companies etc.) and "as such the location data provided may not be accurate". Hence, it might be misleading to assume that customers listed with these postcodes come from the same demographics. In addition, the number of missing values in this section is less than 0.05%. 

Option 2: Impute with median values from a state level since we can only infer the state from postcodes. However, imputation on state level may be too broad, especially considering how most states (except ACT) have similar median values for the various scores, based on initial analysis.

In [11]:
# code to remove rows
start = len(consumerxabs)
missing_postcodes = list(set(tbl_consumer_sa2.loc[tbl_consumer_sa2['SA2_code_2016'].isnull(), 'postcode'].tolist()))
consumerxabs = consumerxabs[~consumerxabs['postcode'].isin(missing_postcodes)]
post_removal = len(consumerxabs)
print(f"removed {start - post_removal} records, or {(start - post_removal)/start*100:.4f}% of records")

removed 488 records, or 0.0499% of records


### Deal with unmatched consumer SA2 codes
Some SA2 codes in the consumer data does not have its corresponding entries in the ABS data. We suspect this was because our joins involved multiple files. The SA2-postcode mapping file is not officially from the ABS website, so it may be inconsistent with the correspondence file and ABS data. Hence, it is quite reasonable to remove the missing values that resulted from this issue, given the small magnitude. In addition, we assume that SA2 areas that are not in the official ABS data are considered to be small and irrelevant areas, hence it makes sense to remove missing values surrounding these areas.

In [12]:
# remove non-existing SA2 codes in the ABS data
consumerxabs_dropped = consumerxabs.dropna()
new_count = len(consumerxabs_dropped)
print(f"removed {post_removal-new_count} rows, or {(post_removal-new_count)/post_removal*100:.4f}% of records")

removed 12495 rows, or 1.2793% of records
